In [ ]:
%pip install q langchain-ollama

In [8]:
%pip install -q langchain

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install -U langchain-community

In [ ]:
%pip install faiss-cpu

In [13]:
%pip install langchain_huggingface

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="phi4:latest", temperature=0.5)

In [5]:
response = llm.invoke("what is langchain?")
print(response)

LangChain is an open-source framework designed to facilitate the development of applications that leverage large language models (LLMs) and other AI technologies. It provides tools and components for building, deploying, and managing AI-driven systems, focusing on modularity and flexibility.

Key features of LangChain include:

1. **Modular Architecture**: Allows developers to easily integrate different components such as LLMs, databases, and user interfaces.
   
2. **Agent Framework**: Supports the creation of agents that can perform complex tasks by chaining together multiple AI models or services.

3. **Memory Management**: Offers tools for managing state and memory across interactions, which is crucial for maintaining context in conversational applications.

4. **Integration with LLMs**: Provides seamless integration with various large language models like OpenAI's GPT series, enabling developers to leverage their capabilities within custom applications.

5. **Customization and Ext

In [15]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [16]:
from langchain.vectorstores import FAISS

# Step 5: Reload the Vector Store
loaded_faiss_store = FAISS.load_local(
    "/workspaces/RAG_BOT/LocalEmbeddings/huggingface_faiss_index",
    embedding_model,
    allow_dangerous_deserialization=True  # Enable deserialization
)
print("FAISS vector store reloaded.")

FAISS vector store reloaded.


In [17]:
from langchain.chains import RetrievalQA

retriever = loaded_faiss_store.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [20]:
# Replace the question with the one you want to ask.
query = "How can I manage file attachments and storage space for secured items?"
print(f"\nQuestion: {query}")

# Retrieve and answer the query
result = qa_chain.invoke({"query": query})
print("\nAnswer:")
print(result["result"])


Question: How can I manage file attachments and storage space for secured items?


ResponseError: model requires more system memory (11.0 GiB) than is available (10.5 GiB) (status code: 500)